In [1]:
import numpy as np
import astropy.io.fits as pyfits

from sys import argv
import warnings
import time
import os

In [2]:
#----------------------------------------
# define functions
#----------------------------------------
def brightness_T(I, bmaj, bmin, freq):
    '''
    Calculate the Brightness Teperature (K)
    '''
    I_mJy_beam = I*1000.              # Stoke I [y/beam -> mJy/beam]
    bmaj_arcs = bmaj*3600.            # maximum axis of beam [deg -> arcsec]
    bmin_arcs = bmin*3600.            # minimum axis of beam [deg -> arcsec]
    freq_GHz = np.divide(freq,1e9)    # Frequency [Hz -> GHz]
    BT_K = 1.222e3*np.divide( I_mJy_beam, np.power(freq_GHz,2)*bmaj_arcs*bmin_arcs )
    
    return BT_K

In [3]:
#---------------------------------------
# Input some parameters from parameter.ipynb
#---------------------------------------
%run parameter.ipynb

#---------------------------------------
# Input parameters used in this program:

###-------------------------------------
### Path name & outname & imageName
# path_image, path_BT
# outname_b4,  outname_b7
# I_imageName, BT_imageName

###-------------------------------------
### criteria and rms for the imaging
# I_clip_b4, I_clip_b7 [Jy beam-1]

In [4]:
%%time
#---------------------------------------
# Set the outname (different observed bands)
outnames = [
             outname_b4, outname_b7
            ]

for outname in outnames:

    #---------------------------------------
    # Get the Imagename
    I_imageName, Q_imageName, U_imageName, PI_imageName, PI_cut3PI_imageName,\
    PA_imageName, Per_imageName, Per_cut3PI_imageName, BT_imageName = slt_imageName(outname)

    # Get the rms and criteria
    rms_I, rms_Q, rms_PI, I_clip, PI_lowclip, PI_highclip, \
    sampling, PI_vmax, convert_unit = slt_parm(outname)

    # Get the FITS and header
    I_data = pyfits.getdata(I_imageName)
    I_hd = pyfits.getheader(I_imageName)

    # Get the frequency and beam information from header
    hd_freq = float(I_hd['RESTFRQ'])
    hd_bmaj = float(I_hd['BMAJ'])
    hd_bmin = float(I_hd['BMIN'])
    hd_bpa = float(I_hd['BPA'])

    print ("{0} frequency = {1} [GHz]".format(outname[12:14], hd_freq/1e9))
    print ("{0} beamsize = {1:.3f} x {2:.3f} [arcsec], {3:.3f} [deg]".format(outname[12:14], hd_bmaj*3600.,hd_bmin*3600.,hd_bpa))
    print ('#----------------------------------------')
    #---------------------------------------
    # Calculate brightness temperature
    BT = brightness_T(I_data, hd_bmaj, hd_bmin, hd_freq)
    BT[I_data<I_clip] = np.nan

    # Save the brightness temperature FITS file
    # caution: header is from Stokes I
    pyfits.writeto(BT_imageName, BT, I_hd, overwrite=True)

B4 frequency = 138.00000000010002 [GHz]
B4 beamsize = 0.200 x 0.100 [arcsec], -0.000 [deg]
#----------------------------------------
B7 frequency = 336.5000000001 [GHz]
B7 beamsize = 0.200 x 0.100 [arcsec], -0.000 [deg]
#----------------------------------------
CPU times: user 164 ms, sys: 321 ms, total: 485 ms
Wall time: 634 ms
